In [1]:
from typing import List
from Dataset.CustomDataset import AgeGroupAndAgeDataset, StandardDataset, AgeGroupAndAgeDatasetKL, AgeGroupKLAndAgeDatasetKL
from Dataset.CustomDataLoaders import CustomDataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from Utils import AAR, CSVUtils, AgeConversion
from Utils.Validator import Validator

c:\Users\Gio\.conda\envs\AV_project\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Caricamento del dataframe
df = CSVUtils.get_df_from_csv("./training_caip_contest.csv", "./training_caip_contest/")

In [3]:
df_train, df_val = train_test_split(df, test_size=0.25, random_state=42)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
#########################
import pandas as pd
df = CSVUtils.get_df_from_csv("./training_caip_contest.csv", "./training_caip_contest/")
df_train, df_val = train_test_split(df, test_size=0.25, random_state=42)
aug = CSVUtils.get_df_from_csv("./augumentation_simple.csv", "./newAugmentationDataset/")
df_train = pd.concat([df_train, aug], ignore_index=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
#########################

from torchvision import transforms
import torch

transform_func = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
    transforms.RandAugment(2, 9),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

transform_func_val = transforms.Compose([
    transforms.Resize(224),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

label_map_v = CSVUtils.get_label_map_vector()
cd_train = AgeGroupKLAndAgeDatasetKL(df_train, path_col="path", label_col="age", label_function="Linear", 
                                    label_map_vector=label_map_v, transform_func=transform_func)

cd_val = StandardDataset(df_val, path_col="path", label_col="age", label_function="CAE", transform_func=transform_func_val)
cd_val.set_n_classes(81)
cd_val.set_starting_class(1)
validator = Validator(cd_val, AgeConversion.EVAge, 32, num_workers=4, prefetch_factor=4)

dm_train = CustomDataLoader(cd_train)
dl_train = dm_train.get_unbalanced_dataloader(batch_size=128, shuffle=True, drop_last=True, num_workers=16, prefetch_factor=4, pin_memory=True)

In [4]:
from ResNetFilmed.resnet import ResNetFiLMed, BackBone
import torch
from torch import optim
import torch.nn.functional as F
from torch import nn

####################################################
EPOCHS = 12
####################################################

backbone = BackBone(pretrained=True, freeze=False)
backbone.train()
model_age_group = ResNetFiLMed(backbone, 8)
model_age_group.train()
model_age = ResNetFiLMed(backbone, 81)
model_age.train()
opt = optim.SGD(set([*model_age_group.parameters(), *model_age.parameters()]), lr=0.1, weight_decay=5e-4)
scheduler = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(dl_train), epochs=EPOCHS, three_phase=True)
kl = nn.KLDivLoss(reduction="batchmean")

In [5]:
best_val_aar = -1
knowledge_age_group = torch.tensor([[0.125]*8]*128, requires_grad=False).float().to("cuda")

for e in range(EPOCHS):
    with tqdm(dl_train, unit=" batch") as tepoch:
        for batch in tepoch:
            opt.zero_grad()
            x, y = batch
            x = x.to("cuda")
            y_age_group = y[0].to("cuda")
            y_age = y[1].to("cuda")
            y_age_kl: torch.Tensor = y[2].to("cuda")
            
            knowledge = model_age_group(x, knowledge_age_group)
            loss_age_group: torch.Tensor = kl(F.log_softmax(knowledge, dim=-1), y_age_group)

            knowledge = F.softmax(knowledge, dim=-1)
            out_age = model_age(x, knowledge)
            loss_age_kl: torch.Tensor = kl(F.log_softmax(out_age, dim=-1), y_age_kl)

            out = F.softmax(out_age, dim=-1)
            out = AgeConversion.EVAge(out).to("cuda")
            loss_age: torch.Tensor = torch.mean(torch.abs(out - y_age))

            loss = loss_age_group + loss_age_kl + loss_age
            loss.backward()
            opt.step()
            scheduler.step()

            tepoch.set_postfix(loss_age_group=loss_age_group.detach().cpu().numpy(), 
                                loss_age=loss_age.detach().cpu().numpy(), 
                                loss_age_kl=loss_age_kl.detach().cpu().numpy())

    def forward_function(x):
        knowledge_age_group = torch.tensor([[0.125]*8]*len(x), requires_grad=False).float().to("cuda")
        knowledge = model_age_group(x, knowledge_age_group)
        knowledge = F.softmax(knowledge, dim=-1)
        out = model_age(x, knowledge)
        out = F.softmax(out, dim=-1)
        return out

    mae, val_aar, val_aar_old = validator.validate_ext(forward_function)
    print(mae, val_aar, val_aar_old)

    if val_aar > best_val_aar:
        best_val_aar = val_aar
        torch.save(model_age_group.state_dict(), "./model_age_group_film_feature.pt")
        torch.save(model_age.state_dict(), "./model_age_film_feature.pt")
        print("Saved model")

100%|██████████| 4493/4493 [03:03<00:00, 24.49 batch/s]


tensor(4.3063, dtype=torch.float64) tensor(0., dtype=torch.float64) tensor(2.6937, dtype=torch.float64)
Saved model


100%|██████████| 4493/4493 [03:20<00:00, 22.42 batch/s]


tensor(3.9628, dtype=torch.float64) tensor(0., dtype=torch.float64) tensor(3.0372, dtype=torch.float64)


100%|██████████| 4493/4493 [03:09<00:00, 23.75 batch/s]


...

tensor(2.4407, dtype=torch.float64) tensor(2.8013, dtype=torch.float64) tensor(4.5593, dtype=torch.float64)


100%|██████████| 4493/4493 [02:44<00:00, 27.32 batch/s]


tensor(2.2242, dtype=torch.float64) tensor(3.7748, dtype=torch.float64) tensor(5.0831, dtype=torch.float64)
Saved model


100%|██████████| 4493/4493 [02:45<00:00, 27.17 batch/s]


tensor(2.1877, dtype=torch.float64) tensor(4.1028, dtype=torch.float64) tensor(5.3027, dtype=torch.float64)
Saved model


In [5]:
####################################################
EPOCHS = 12
####################################################

dl_train = dm_train.get_balanced_class_dataloader(class_ranges=[(0, 11), (11, 21), (21, 31), (31, 41), (41, 51), (51, 61), (61, 71), (71, 91)], 
                                                  batch_size=128, num_workers=8, prefetch_factor=4, pin_memory=True)

model_age_group.load_state_dict(torch.load("./model_age_group_film_feature_mae2.pt"))
model_age_group.requires_grad_(False)
model_age_group.fc0.requires_grad_(True)
model_age.load_state_dict(torch.load("./model_age_film_feature_mae2.pt"))
opt = optim.SGD(set([*model_age_group.fc0.parameters()]), lr=0.1, weight_decay=5e-4)
scheduler = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(dl_train), epochs=EPOCHS, three_phase=True)

In [6]:
def forward_function(x):
    knowledge_age_group = torch.tensor([[0.125]*8]*len(x), requires_grad=False).float().to("cuda")
    knowledge = model_age_group(x, knowledge_age_group)
    knowledge = F.softmax(knowledge, dim=-1)
    out = model_age(x, knowledge)
    out = F.softmax(out, dim=-1)
    return out

mae, val_aar, val_aar_old = validator.validate_ext(forward_function)
print(mae, val_aar, val_aar_old)

100%|██████████| 4493/4493 [04:05<00:00, 18.28 batch/s]


tensor(2.1877, dtype=torch.float64) tensor(4.1033, dtype=torch.float64) tensor(5.2903, dtype=torch.float64)


In [7]:
best_val_aar = val_aar
knowledge_age_group = torch.tensor([[0.125]*8]*128, requires_grad=False).float().to("cuda")

for e in range(EPOCHS):
    with tqdm(dl_train, unit=" batch") as tepoch:
        for batch in tepoch:
            opt.zero_grad()
            x, y = batch
            x = x.to("cuda")
            y_age_group = y[0].to("cuda")
            y_age = y[1].to("cuda")
            y_age_kl: torch.Tensor = y[2].to("cuda")
            
            knowledge = model_age_group.forward_detach(x, knowledge_age_group)
            #loss_age_group: torch.Tensor = kl(F.log_softmax(knowledge, dim=-1), y_age_group)

            knowledge = F.softmax(knowledge, dim=-1)
            out_age = model_age(x, knowledge)
            loss_age_kl: torch.Tensor = kl(F.log_softmax(out_age, dim=-1), y_age_kl)

            out = F.softmax(out_age, dim=-1)
            out = AgeConversion.EVAge(out).to("cuda")
            loss_age: torch.Tensor = torch.mean(torch.abs(out - y_age))

            loss = loss_age_kl + (torch.square(loss_age - mae) if loss_age > mae else torch.tensor(0)) # + loss_age_group + loss_age_kl
            loss.backward()
            opt.step()
            scheduler.step()

            tepoch.set_postfix(#loss_age_group=loss_age_group.detach().cpu().numpy(), 
                                loss_age=loss_age.detach().cpu().numpy(),
                                loss_age_kl=loss_age_kl.detach().cpu().numpy())

    def forward_function(x):
        knowledge_age_group = torch.tensor([[0.125]*8]*len(x), requires_grad=False).float().to("cuda")
        knowledge = model_age_group(x, knowledge_age_group)
        knowledge = F.softmax(knowledge, dim=-1)
        out = model_age(x, knowledge)
        out = F.softmax(out, dim=-1)
        return out

    mae_, val_aar, val_aar_old = validator.validate_ext(forward_function)
    print(mae_, val_aar, val_aar_old)

    if val_aar > best_val_aar:
        best_val_aar = val_aar
        torch.save(model_age_group.state_dict(), "./model_age_group_film_classifier_middle.pt")
        torch.save(model_age.state_dict(), "./model_age_film_classifier_middle.pt")
        print("Saved model")

100%|██████████| 4493/4493 [03:40<00:00, 20.41 batch/s]


tensor(2.7673, dtype=torch.float64) tensor(4.5907, dtype=torch.float64) tensor(5.4681, dtype=torch.float64)
Saved model


100%|██████████| 4493/4493 [03:37<00:00, 20.62 batch/s]


tensor(2.5560, dtype=torch.float64) tensor(4.3494, dtype=torch.float64) tensor(5.4120, dtype=torch.float64)


100%|██████████| 4493/4493 [03:54<00:00, 19.19 batch/s]


...

tensor(2.7055, dtype=torch.float64) tensor(4.8900, dtype=torch.float64) tensor(5.6563, dtype=torch.float64)
Saved model


100%|██████████| 4493/4493 [03:38<00:00, 20.54 batch/s]


tensor(2.7004, dtype=torch.float64) tensor(4.9417, dtype=torch.float64) tensor(5.6821, dtype=torch.float64)
Saved model


100%|██████████| 4493/4493 [03:49<00:00, 19.55 batch/s]


tensor(2.6964, dtype=torch.float64) tensor(4.9640, dtype=torch.float64) tensor(5.6978, dtype=torch.float64)
Saved model
